In [3]:
!pip install pyngrok flatlib plotly numpy

In [5]:
!ngrok config add-authtoken 2roAQEoHDGIzQfgUcMwBMm69QUf_2xjKVe2ibjEAM6H2VwQ9L

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from flatlib import chart, const
from flatlib.datetime import Datetime, Time
from flatlib.geopos import GeoPos
import plotly.graph_objects as go
import numpy as np
import json

app = Flask(__name__)

# Endpoint 1: Generate Birth Chart
@app.route('/api/birth-chart', methods=['POST'])
def birth_chart():
    data = request.json
    # Extract parameters
    birth_date = data['birth_date']
    birth_time = data['birth_time']
    birth_lat = float(data['birth_lat'])
    birth_lon = float(data['birth_lon'])

    # Generate Flatlib chart
    birth_datetime = Datetime(birth_date.replace('-', '/'), Time(birth_time))
    birth_location = GeoPos(birth_lat, birth_lon)
    birth_chart = chart.Chart(birth_datetime, birth_location)

    # Prepare chart data
    houses = [(house.id, house.sign, house.lon) for house in birth_chart.houses]
    planets = [(obj.id, obj.sign, obj.signlon) for obj in birth_chart.objects]

    result = {
        "houses": houses,
        "planets": planets
    }

    return jsonify(result)

# Endpoint 2: Generate 12-House Chart Visualization
@app.route('/api/house-chart', methods=['POST'])
def house_chart():
    # Define house descriptions
    houses_descriptions = [
        "Self Appearance, Beginnings, Identity, Approach to Life",
        "Money, Work, Income, Values, Material Possessions",
        "The Mind, Communication, Siblings, Neighbors",
        "Home, Roots, Family, Self-Care, Emotions",
        "Romance, Love, Creativity, Fertility, Joy",
        "Health, Fitness, Work, Pets, Organization",
        "Relationships, Marriage, Partnerships, Sharing",
        "Sex, Intimacy, Shared Finances, Transformation",
        "Travel, Wisdom, Philosophy, Higher Education",
        "Career, Status, Reputation, Fame, Public Image",
        "Friends, Groups, Hopes, Humanitarianism",
        "Endings, Closure, Solitude, Spirituality, Subconscious",
    ]

    # Angular positions for the houses
    angles = np.linspace(0, 360, 13)
    angle_midpoints = [(angles[i] + angles[i+1]) / 2 for i in range(12)]

    # Radial positions and colors
    r_values = [0.5, 1]
    colors = [
        "rgba(255, 99, 71, 0.5)",  # Light Coral
        "rgba(135, 206, 250, 0.5)",  # Light Sky Blue
        "rgba(144, 238, 144, 0.5)",  # Light Green
        "rgba(255, 182, 193, 0.5)",  # Light Pink
        "rgba(255, 215, 0, 0.5)",  # Gold
        "rgba(173, 216, 230, 0.5)",  # Light Blue
        "rgba(240, 128, 128, 0.5)",  # Light Salmon
        "rgba(221, 160, 221, 0.5)",  # Plum
        "rgba(176, 196, 222, 0.5)",  # Light Steel Blue
        "rgba(255, 228, 181, 0.5)",  # Moccasin
        "rgba(152, 251, 152, 0.5)",  # Pale Green
        "rgba(240, 230, 140, 0.5)",  # Khaki
    ]

    # Create Plotly figure
    fig = go.Figure()
    for i, desc in enumerate(houses_descriptions):
        fig.add_trace(go.Scatterpolar(
            r=[r_values[0], r_values[1], r_values[1], r_values[0], r_values[0]],
            theta=[angles[i], angles[i], angles[i+1], angles[i+1], angles[i]],
            fill='toself',
            fillcolor=colors[i],
            line=dict(color='black', width=1),
            name=f"House {i+1}",
            hoverinfo='text',
            text=f"House {i+1}<br>{desc}"
        ))

    # Add house labels
    for i, midpoint in enumerate(angle_midpoints):
        fig.add_trace(go.Scatterpolar(
            r=[1.1],
            theta=[midpoint],
            mode='text',
            text=f"{i+1}",
            textposition="top center",
            textfont=dict(size=12, color='black'),
            hoverinfo='skip'
        ))

    # Configure layout
    fig.update_layout(
        polar=dict(
            angularaxis=dict(visible=False),
            radialaxis=dict(visible=False)
        ),
        title="12-House Birth Chart",
    )

    # Return the chart as JSON
    return jsonify(fig.to_plotly_json())

# Start the Flask app and ngrok tunnel
if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")
    app.run()

Public URL: NgrokTunnel: "https://6960-34-106-4-160.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 16:38:13] "POST /api/birth-chart HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 16:38:22] "POST /api/house-chart HTTP/1.1" 200 -
